In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

import pandas as pd
import numpy as np
from fbi_nibrs import *
import time
from states import *

# # state = states_av[1]
# path = "/root/Work/cdc_project/FBI_UCR/data/compressed"
# files = os.listdir(path)
# files.sort()
# # years = ["2012"]#, "2016", "2017", "2018", "2019", "2020"]

# stats = {
#     "year": [],
#     "state": [],
#     "incidents": [],
#     "CM_Other": [],
#     "CM_Physical_Abuse": [],
#     "CM_Neglect": [],
#     "CM_Sexual_Abuse": [],
#     "IPV_Other": [],
#     "IPV_Physical_Abuse": [],
#     "IPV_Neglect_or_Emotional_Abuse": [],
#     "IPV_Sexual_Abuse": [],
# }

# outcomes = pd.DataFrame()
# corruped_files = []

# for state in states_av:
#     files_state = [path + '/' + file for file in files if state in file]
#     print(f"\n {state}", end=" ")
#     for zip_file in files_state:
#         if zip_file[-4:] != ".zip":
#             os.remove(zip_file)
#         print(f"{zip_file[-8:-4]}", end=" ")
#         temp_folder = "/root/Work/cdc_project/FBI_UCR/data/temp/"
#         temp_folder = extract_files(zip_file, temp_folder, state, keep_files=False)
#         try:
#             df = filter_data(temp_folder)
#             stats["year"].append(zip_file[-8:-4])
#             stats["state"].append(state)
#             stats["incidents"].append(len(df))
#             if len(df) == 0:
#                 stats["total_CM_Neglect"].append(0)
#                 stats["total_CM_Sexual_Abuse"].append(0)
#                 stats["total_IPV"].append(0)
#                 stats["total_IPV_Physical_Abuse"].append(0)
#                 stats["total_IPV_Neglect_or_Emotional_Abuse"].append(0)
#                 stats["total_IPV_Sexual_Abuse"].append(0)
#             else:   
#                 # Tag the type of incident
#                 df["OUTCOME"] = df.apply(lambda x : get_crime_type(x.RELATIONSHIP_ID, x.OFFENSE_TYPE_ID), axis=1)
#                 outcomes = outcomes.append(df)
#                 counts = df.OUTCOME.value_counts()
#                 for key in stats.keys():
#                     if key in ["year", "state", "incidents"]:
#                         continue
#                     if key in counts.index:
#                         stats[key].append(counts[key])
#                     else:
#                         stats[key].append(0)
#         except:
#             temp_folder = remove_files(temp_folder, state)
#             print(u'\u2717', end=" ")
#             corruped_files.append(zip_file)
#             stats["year"].pop()
#             stats["state"].pop()
#             stats["incidents"].pop()
#             continue
#         # break
#         temp_folder = remove_files(temp_folder, state)
#         print(u'\u2713', end=" ")

# stats_df = pd.DataFrame(stats)
# stats_df.to_csv("/root/Work/cdc_project/FBI_UCR/data/stats.csv", index=False)
# outcomes.to_csv("/root/Work/cdc_project/FBI_UCR/data/outcomes.csv", index=False)
# corruped_files_series = pd.Series(corruped_files)
# corruped_files_series.to_csv("/root/Work/cdc_project/FBI_UCR/data/corruped_files.csv", index=False)

In [3]:
states = {value:key for key, value in states_name_dict.items()}

stats_df = pd.read_csv("/root/Work/cdc_project/FBI_UCR/data/stats.csv")

agency_participation = pd.read_csv("/root/Work/cdc_project/FBI_UCR/data/ucr_participation_1960_2020.csv")
agency_participation.rename(columns={"state_name": "state", "data_year": "year"}, inplace=True)
agency_participation.state = agency_participation.state.apply(lambda x: states[x] if x in states.keys() else np.nan)
agency_participation.dropna(subset=["state"], inplace=True)

#  Remome observations outisde the range of the years [2000, 2020]
stats_df = stats_df[(stats_df.year >= 2000) & (stats_df.year <= 2020)]
agency_participation = agency_participation[(agency_participation.year >= 2000) & (agency_participation.year <= 2020)]

# # Identify the states with consecutive years of data
states_consecutive = stats_df.groupby("state").agg({"year": lambda x: x.diff().sum() == 20})
states_consecutive = states_consecutive[states_consecutive.year ].index.tolist()

# Remove the states without consecutive years of data
stats_df = stats_df[stats_df.state.isin(states_consecutive)]
agency_participation = agency_participation[agency_participation.state.isin(states_consecutive)]

agency_participation.head()

stats_df = stats_df.merge(agency_participation[["year", "state", "total_population", "nibrs_population"]], on=["state", "year"], how="left")
stats_df["pct_population_covered"] = stats_df["nibrs_population"] / stats_df["total_population"]


stats_df.loc[:, "incidents_normalized"] = stats_df.incidents / stats_df.nibrs_population # normalize by population
stats_df.loc[:, "CM_normalized"] = stats_df[ ["CM_Other", "CM_Physical_Abuse", "CM_Neglect", "CM_Sexual_Abuse"]].sum(axis=1) / stats_df.nibrs_population # normalize by population
stats_df.loc[:, "IPV_normalized"] = stats_df[["IPV_Other", "IPV_Physical_Abuse", "IPV_Neglect_or_Emotional_Abuse", "IPV_Sexual_Abuse"]].sum(axis=1) / stats_df.nibrs_population # normalize by population

stats_df.to_csv("/root/Work/cdc_project/FBI_UCR/data/stats_with_population.csv", index=False)

In [12]:
# States 80% of the population are covered by NIBRS
states80 = []
for g, d in stats_df.groupby("state"):
    if d.pct_population_covered.min() >= 0.80:
        states80.append(g)

In [13]:
stats_df[stats_df.state.isin(states80)]#.to_csv("/root/Work/cdc_project/FBI_UCR/data/stats_80_pct_population_covered.csv", index=False)

,year,state,incidents,CM_Other,CM_Physical_Abuse,CM_Neglect,CM_Sexual_Abuse,IPV_Other,IPV_Physical_Abuse,IPV_Neglect_or_Emotional_Abuse,IPV_Sexual_Abuse,total_population,nibrs_population,pct_population_covered,incidents_normalized,CM_normalized,IPV_normalized
82,2000,IA,1994,0,239,0,73,30,1596,0,56,3153624,2840965,0.900857,0.000702,0.000110,0.000592
83,2001,IA,1911,0,158,0,68,34,1587,0,64,3199087,2861030,0.894327,0.000668,0.000079,0.000589
84,2002,IA,1941,0,154,0,80,37,1604,0,66,3146200,2864063,0.910325,0.000678,0.000082,0.000596
85,2003,IA,1965,0,197,0,60,41,1597,0,70,3193360,2916051,0.913161,0.000674,0.000088,0.000586
86,2004,IA,2035,0,167,0,76,33,1670,0,89,3147746,2914838,0.926008,0.000698,0.000083,0.000615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,2016,VT,279,0,25,0,8,4,216,0,26,727124,625702,0.860516,0.000446,0.000053,0.000393
414,2017,VT,342,0,33,0,14,5,261,0,29,636517,623657,0.979796,0.000548,0.000075,0.000473
415,2018,VT,428,0,45,0,34,3,301,0,45,648985,630980,0.972257,0.000678,0.000125,0.000553
416,2019,VT,426,0,52,0,10,3,325,0,36,652209,628664,0.963900,0.000678,0.000099,0.000579
